In [ ]:
import numpy as np
import cv2
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import os

In [ ]:
vehicle_folder = '/content/drive/MyDrive/vehicle/vehicles'
non_vehicle_folder = '/content/drive/MyDrive/vehicle/non-vehicles'

vehicle_images = []
non_vehicle_images = []

for filename in os.listdir(vehicle_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = cv2.imread(os.path.join(vehicle_folder, filename))
        if img is not None:
            vehicle_images.append(img)

for filename in os.listdir(non_vehicle_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = cv2.imread(os.path.join(non_vehicle_folder, filename))
        if img is not None:
            non_vehicle_images.append(img)

In [ ]:
def extract_hog_features(images):
    hog_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor()
        features = hog.compute(gray)
        hog_features.append(features.flatten())
    return hog_features

In [ ]:
vehicle_features = extract_hog_features(vehicle_images)
non_vehicle_features = extract_hog_features(non_vehicle_images)

In [ ]:
X = np.concatenate((vehicle_features, non_vehicle_features))
y = np.concatenate((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
svm_model = svm.SVC(C=10, kernel='linear')
svm_model.fit(X_train, y_train)

SVC(C=10, kernel='linear')

In [ ]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9504504504504504


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly']
}

svm_model = svm.SVC()

grid_search = GridSearchCV(svm_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_svm_model = grid_search.best_estimator_

best_svm_model.fit(X_train, y_train)

SVC(C=10)

In [ ]:
joblib.dump(svm_model, 'svm.pkl')

['svm.pkl']

In [ ]:
model = joblib.load('svm.pkl')